In [ ]:
require('rnn')
require('hdf5')
require('optim')


f = hdf5.open("data.hdf5", "r")



train_rnn_X = f:read('train_rnn_X'):all()
train_rnn_Y = f:read('train_rnn_Y'):all()
train_X_sequence =  f:read('train_X_sequence'):all()
train_Y  = f:read('train_Y'):all()
valid_reduced_X = f:read('valid_reduced_X'):all()
valid_reduced_Y = f:read('valid_reduced_Y'):all()
windows_train = f:read('windows_train'):all()
windows_valid = f:read('windows_valid'):all()
test = f:read('test'):all()
valid_kaggle_with_spaces = f:read('valid_kaggle_with_spaces'):all()
valid_kaggle_without_spaces = f:read('valid_kaggle_without_spaces'):all()
valid_answers = f:read('valid_answers'):all()
nfeatures = (f:read('nfeatures'):all())[1]
nclasses = (f:read('nclasses'):all())[1]




f:close()

window_size = windows_train:size(2)
n_actual = nclasses

In [ ]:
vocab_size = nfeatures
embed_dim = 15
lin_dim = 100
LT = nn.LookupTable(vocab_size, embed_dim)

lstm = nn.Sequential()
lstm:add(LT)
--lstm:add(nn.Dropout())
lstm:add(nn.SplitTable(1, 3))
seqLSTM = nn.Sequencer(nn.LSTM(embed_dim, lin_dim))
seqLSTM:add(nn.Dropout())
seqLSTM:remember('both')
lstm:add(seqLSTM)

lstm:add(nn.Sequencer(nn.Linear(lin_dim, n_actual)))

--lstm:add(nn.Sequencer(nn.LogSoftMax()))

--crit = nn.SequencerCriterion(nn.BCECriterion())

crit = nn.SequencerCriterion(nn.CrossEntropyCriterion())
params, grad_params = lstm:getParameters()
params:rand(params:size()):csub(.5):mul(.05)
count = 0

In [ ]:
seqLSTM:forget()
count = 0
function feval(w_new)
 
    --count = count + 1
    count = math.fmod(count + 1,train_rnn_X:size(1))
    if count == 0 then
        count = 1
        seqLSTM:forget()
    end
    
    x_d = (train_rnn_X[count]):t()
    y_d = (train_rnn_Y[count]):t()
    
    
    grad_params:zero()
    preds = lstm:forward(x_d)
    --print(preds[1])
    loss = crit:forward(preds, y_d)
    grad_out = crit:backward(preds, y_d)
    
    lstm:backward(x_d, grad_out)
    
    if torch.norm(grad_params) > 5 then
        nn.Normalize(2):forward(grad_params)
        grad_params:mul(5)
    end
    
    return loss, grad_params
end

for i = 1,1000 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,params)
    if i % 100 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] , count)
        
        -- print(sgd_params)
    end
end

In [ ]:
function perplexity(input_set, output_set)
    sum = 0
    seqLSTM:forget()
    distributions = torch.Tensor(input_set:size(1),2)
    for i = 1,input_set:size(1) do
        
        a = lstm:forward(input_set:sub(i,i))
        
        distribution = (nn.LogSoftMax():forward(a[1]))
        
        distributions[i] = torch.exp(distribution)
        answer = distribution[output_set[i]]
        sum = sum + answer
        
    end
    return torch.exp(-sum/input_set:size(1)), distributions
end

In [ ]:
a,b = perplexity((valid_reduced_X:sub(1,10000)), valid_reduced_Y)
print(a)


In [ ]:
a,b = perplexity((train_X_sequence:sub(1,10000)), train_Y:sub(1,100000))
print(a)


In [ ]:
function predict(input_set, output_set, cutoff)
    kaggle = torch.zeros(input_set:size(1))
    for i = 1,input_set:size(1) do
        count = 0
        lstm:forget()
        for j = 1, input_set:size(2) do
            if input_set[i][j] == 30 then
                
                break
            end
            a = lstm:forward(torch.Tensor{input_set[i][j]})
            
            distribution = (nn.LogSoftMax():forward(a[1]))
            distribution = torch.exp(distribution)
            
            if distribution[2] > cutoff then
                space = lstm:forward(torch.Tensor{28})
                count = count + 1
            end
            
        
        end
        kaggle[i] = count
    
    end
    if output_set ~= 0 then
        out =output_set:double()
        MSE = nn.MSECriterion():forward(kaggle, out)
    else
        out = 0
        MSE = 0
    end
    
    return kaggle, out, MSE
end

In [ ]:
k, o, mse = predict(valid_kaggle_without_spaces, valid_answers,.24)
print(mse)

In [ ]:
k,_,_ = predict(test, 0, .24)

In [ ]:
function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Count\n")
    for i=1,scores:size(1) do
        s = tostring(i) .. "," .. tostring(scores[i])
        f:write(s .. "\n")
    end
    f:close()
end

write2file(k, 'preds.csv')